In [ ]:
%pip install transformers

In [ ]:
%pip install datasets

In [ ]:
%pip install evaluate

In [ ]:
%pip install openai

In [ ]:
import openai
import csv
from time import sleep

input_file = 'output.csv'
output_file = 'output_ranking.csv'

openai.api_key = "YOUR_OPENAI_KEY"

def prompt_ranking(reference_text,output1_text,output2_text,output3_text,output4_text,output5_text,output6_text,output7_text):
  prompt_template = """
  Reference: {reference}

  output1: {output1}

  output2: {output2}

  output3: {output3}

  output4: {output4}

  output5: {output5}

  output6: {output6}

  output7: {output7}
  
  According to the facts of the disease and the drug and test recommendations in reference output, order the fact match of the output from highest to lowest.

  Strictly follow the following output format：
  
  1. Output1
  2. Output5
  3. Output2
  4. Output6
  5. Output3
  6. Output7
  7. Output4
  """

  prompt = prompt_template.format(reference=reference_text, output1=output1_text, output2=output2_text, output3=output3_text, output4=output4_text, output5=output5_text, output6=output6_text,output7=output7_text)
  response = openai.ChatCompletion.create(
      model="gpt-4",
      messages=[
          {"role": "system", "content": "You are a excellent."},
          {"role": "user", "content": prompt}
      ]
  )

  generated_text = response.choices[0].message.content
  return generated_text

with open(input_file,'r',newline="") as f_input, open(output_file, 'a+', newline='') as f_output:
  reader = csv.reader(f_input)
  writer = csv.writer(f_output)

  header = next(reader)
  header.append("gpt4_ranking")
  writer.writerow(header)

  for row in reader:
    reference_text = row[1].strip("\n")
    output1_text = row[2].strip("\n") 
    output2_text = row[3].strip("\n") 
    output3_text = row[4].strip("\n") 
    output4_text = row[5].strip("\n") 
    output5_text = row[6].strip("\n") 
    output6_text = row[7].strip("\n") 
    output7_text = row[8].strip("\n") 

    flag = 0
    while flag == 0:
      try:
        str1 = prompt_ranking(reference_text,output1_text,output2_text,output3_text,output4_text,output5_text,output6_text,output7_text)
        flag = 1
      except:
        sleep(40)
        str1 = prompt_ranking(reference_text,output1_text,output2_text,output3_text,output4_text,output5_text,output6_text,output7_text)

    print(str1)
    print('\n')

    row.append(str1)
    writer.writerow(row)


In [ ]:
import re
import csv

input_file = 'output_ranking.csv'
output_file = 'output_ranking_compute.csv'

with open(input_file, 'r', newline="") as f_input, open(output_file, 'a+', newline='') as f_output:
    reader = csv.reader(f_input)
    writer = csv.writer(f_output)

    header = next(reader)
    header.extend(["output1_ranking", "output2_ranking", "output3_ranking", "output4_ranking", "output5_ranking", "output6_ranking", "output7_ranking"])
    writer.writerow(header)

    for row in reader:
        output_text = row[9]
        print(output_text)
        matches = re.findall(r'(?m)^\d+\. (\w+):', output_text)

        if matches == []:
            matches = re.findall(r'(?m)^\d+\. (\w+)', output_text)

        if matches == []:
            matches = re.findall(r'\d+\.\s+(\w+)', output_text)
        else:
            if matches[0] == "Output":
                matches = re.findall(r'(?m)^\d+\.\s*(Output \d+):', output_text)

                if matches == []:
                    matches = re.findall(r'\d+\.\s+(Output\s*\d+)', output_text)

        rankings = {}
        for i, match in enumerate(matches):
            rank = i + 1
            output = match.replace(" ", "").lower()  
            rankings[output] = rank

        output_dic = {}

        for i in range(1, 8):
            output_key = f"output{i}".lower()  
            output_dic[output_key] = rankings.get(output_key, None) 

        row.extend([output_dic.get('output1', ''), output_dic.get('output2', ''), output_dic.get('output3', ''), 
                   output_dic.get('output4', ''), output_dic.get('output5', ''), output_dic.get('output6', ''), 
                   output_dic.get('output7', '')])

        writer.writerow(row)



In [ ]:
import pandas as pd

def calculate_column_average(file_path, column_name):
    df = pd.read_csv(file_path)
    average = df[column_name].mean()
    return average, len(df)  

column = ["output1_ranking", "output2_ranking", "output3_ranking", "output4_ranking",
          "output5_ranking", "output6_ranking", "output7_ranking"]

file_path = 'output_ranking_compute.csv'

total_rows = None  
for column_name in column:
    average, row_count = calculate_column_average(file_path, column_name)
    if total_rows is None:
        total_rows = row_count  
    print(f"The average of column {column_name} is: {average}")

print(f"Total number of rows in the file: {total_rows}")